# API Quest
## Oslo

In [5]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv

In [6]:
load_dotenv()
token = os.getenv('TOKEN')
print(token)

test


# HYPOTHESIS
- Rich countries have more Nobel Prizes
    - Nobel prize winners immigrate towards rich countries
    - Nobel prize winners immigrate towards stable countries
- Countries of birth / early education have more impact than countries of higher education
- Nobel Prizes Laureates are getting younger
- Nobel Prizes are awarded more to international teams than before

- Gender Differences: Is there a significant difference in the gender ratio among Nobel Prize winners? Has this changed over time?
- Geographic Distribution: In which countries or regions are Nobel Prize winners predominantly located? Has this distribution changed over time?
- Age of Winners: What is the age distribution of Nobel Prize winners? Are there any noticeable trends in age?
- Publications: Are there specific journals where Nobel Prize winners’ research is commonly published? How influential are these journals?

## HYPOTHESIS 1
- 

# DATA SOURCES

1. **Nobel Laureates Data**
	- **Nobel Prize Official Data**
	  - Description: Comprehensive information on all Nobel laureates, including their age, nationality, affiliation, prize category, and motivation.
	  - Link: [Nobel Prize Official Website](https://www.nobelprize.org/organization/developer-zone-2/)
	  - API: [Nobel Prize API](https://www.nobelprize.org/organization/developer-zone-2/)
	- **Kaggle Nobel Laureates Dataset**
	  - Description: A dataset compiled from the Nobel Prize official data, available in CSV format for easy analysis.
	  - Link: [Kaggle Nobel Prize Dataset](https://www.kaggle.com/datasets/imdevskp/nobel-prize/data)

2. **Research and Development (R&D) Expenditure**
	- **World Bank Open Data**
	  - Description: Data on R&D expenditure (% of GDP) by country.
	  - Link: [World Bank R&D Expenditure](https://data.worldbank.org/indicator/GB.XPD.RSDV.GD.ZS)
	- **UNESCO Institute for Statistics (UIS)**
	  - Description: Statistics on gross domestic expenditure on R&D, researchers per million inhabitants, and more.
	  - Link: [UNESCO UIS Data Centre](http://data.uis.unesco.org/)

3. **Economic Indicators**
	- **World Bank GDP Data**
	  - Description: GDP per capita and other economic indicators for countries worldwide.
	  - Link: [World Bank GDP per Capita](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
	- **Heritage Foundation Index of Economic Freedom**
	  - Description: Measures economic freedom in countries across 12 quantitative and qualitative factors.
	  - Link: [Index of Economic Freedom](https://www.heritage.org/index/)

4. **Education Expenditure and Statistics**
	- **UNESCO Education Data**
	  - Description: Data on government expenditure on education as a percentage of GDP and total government expenditure.
	  - Link: [UNESCO Education Expenditure](http://data.uis.unesco.org/)
	- **OECD Education Statistics**
	  - Description: Detailed statistics on education spending, enrollment rates, and educational attainment among OECD countries.
	  - Link: [OECD Education at a Glance](https://www.oecd.org/education/education-at-a-glance/)

5. **Gender Statistics**
	- **UNESCO Gender Parity Index**
	  - Description: Data on gender parity in education and literacy rates.
	  - Link: [UNESCO Gender Equality Data](http://data.uis.unesco.org/)
	- **World Bank Gender Data Portal**
	  - Description: Comprehensive data on gender equality indicators globally.
	  - Link: [World Bank Gender Data](https://datatopics.worldbank.org/gender/)

6. **University Rankings and Affiliations**
	- **Times Higher Education World University Rankings**
	  - Description: Rankings and data on the world’s leading universities.
	  - Link: [THE World University Rankings](https://www.timeshighereducation.com/world-university-rankings)
	- **QS World University Rankings**
	  - Description: Global university rankings based on academic reputation, employer reputation, and faculty/student ratio.
	  - Link: [QS World University Rankings](https://www.topuniversities.com/qs-world-university-rankings)

7. **Political Stability and Governance**
	- **World Bank Worldwide Governance Indicators (WGI)**
	  - Description: Data on political stability, government effectiveness, regulatory quality, and more.
	  - Link: [World Bank WGI](https://info.worldbank.org/governance/wgi/)

8. **Research Publications and Collaboration**
	- **Microsoft Academic Graph**
	  - Description: A comprehensive dataset of scholarly publications, authors, affiliations, and citation relationships.
	  - Link: [Microsoft Academic Graph](https://www.microsoft.com/en-us/research/project/microsoft-academic-graph/)
	- **OpenAlex**
	  - Description: An open catalog of the global research system, including publications, authors, and institutions.
	  - Link: [OpenAlex](https://openalex.org/)

9. **Urbanization and Population Data**
	- **United Nations World Urbanization Prospects**
	  - Description: Data on urban and rural population distributions and trends.
	  - Link: [UN Urbanization Prospects](https://population.un.org/wup/)
	- **World Bank Urban Population Data**
	  - Description: Percentage of the population living in urban areas by country.
	  - Link: [World Bank Urban Population](https://data.worldbank.org/indicator/SP.URB.TOTL.IN.ZS)

10. **Language and Literature Data**
	 - **Ethnologue Languages of the World**
		- Description: Information on the number of speakers of different languages worldwide.
		- Link: [Ethnologue](https://www.ethnologue.com/)
	 - **UNESCO World Atlas of Languages**
		- Description: Data on language usage, vitality, and diversity.
		- Link: [UNESCO Language Atlas](https://en.unesco.org/languages-atlas)

11. **Technology Adoption and Internet Usage**
	 - **International Telecommunication Union (ITU) ICT Statistics**
		- Description: Data on ICT development, including internet users, broadband subscriptions, and mobile cellular subscriptions.
		- Link: [ITU ICT Statistics](https://www.itu.int/en/ITU-D/Statistics/Pages/stat/default.aspx)
	 - **World Bank Technology Adoption Indicators**
		- Description: Statistics on technological infrastructure and adoption levels across countries.
		- Link: [World Bank ICT Data](https://data.worldbank.org/indicator/IT.NET.USER.ZS)

12. **Researcher Mobility and International Collaboration**
	 - **ORCID Public Data File**
		- Description: Contains information on researchers’ affiliations and publication records.
		- Link: [ORCID Public Data File](https://orcid.org/content/orcid-public-data-file)
	 - **Scopus Data via API**
		- Description: Bibliometric data that can be used to analyze collaboration networks (access may be limited).
		- Link: [Scopus APIs](https://dev.elsevier.com/sc_apis.html)

13. **Mentorship and Academic Lineage**
	 - **Mathematics Genealogy Project**
		- Description: Database documenting academic mentors and students, primarily in mathematics.
		- Link: [Mathematics Genealogy Project](https://www.genealogy.math.ndsu.nodak.edu/)
	 - **Academic Family Tree**
		- Description: A broader database of academic mentorship relationships across various disciplines.
		- Link: [Academic Family Tree](https://academictree.org/)

14. **Science, Technology, and Innovation Policies**
	 - **OECD Science, Technology and Innovation Outlook**
		- Description: Analysis and data on global science, technology, and innovation policies.
		- Link: [OECD STI Outlook](https://www.oecd.org/sti/oecd-science-technology-and-innovation-outlook-25186167.htm)
	 - **UNESCO Science Report**
		- Description: Insights into global trends in science governance and policy.
		- Link: [UNESCO Science Report](https://en.unesco.org/node/252277)

15. **Global Health and Human Development**
	 - **UNDP Human Development Reports**
		- Description: Data on human development indices, education levels, and standard of living.
		- Link: [UNDP Human Development Data](http://hdr.undp.org/en/data)
	 - **World Health Organization (WHO) Global Health Observatory**
		- Description: Health-related statistics that can correlate with educational outcomes.
		- Link: [WHO GHO Data](https://www.who.int/data/gho)

16. **Innovation and Patent Data**
	 - **World Intellectual Property Organization (WIPO) Statistics Data Center**
		- Description: Data on patents, trademarks, industrial designs, and other IP statistics.
		- Link: [WIPO IP Statistics](https://www.wipo.int/ipstats/en/)

17. **Data on Developing Countries**
	 - **UN Data**
		- Description: A wide array of statistics on socioeconomic indicators for developing countries.
		- Link: [UN Data](http://data.un.org/)
	 - **World Bank Development Indicators**
		- Description: Comprehensive data on development indicators, including education and economic performance.
		- Link: [World Bank Development Indicators](https://databank.worldbank.org/source/world-development-indicators)

18. **Demographic and Socioeconomic Data**
	 - **United Nations Educational, Scientific and Cultural Organization (UNESCO) Institute for Statistics**
		- Description: Data on education, literacy rates, and demographic factors.
		- Link: [UNESCO UIS Data](http://data.uis.unesco.org/)
	 - **OECD Social and Welfare Statistics**
		- Description: Indicators on social protection, income inequality, and more.
		- Link: [OECD Social Data](https://www.oecd.org/social/soc/)

